In [1]:
%cd ../..
%load_ext autoreload

%autoreload 1

/home/kikohs/work/rts


In [2]:
import os
from pathlib import Path
import rts
import rts.features.pose as pose
import numpy as np

/home/kikohs/.cache/pypoetry/virtualenvs/rts-DIiCth0j-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pose_data = 'data/pifpaf'
videos = '/mnt/d/Videos/EPFL'
%ls $videos

basketball.mp4*  gym.mp4*  running.mp4*  swimming.mp4*  weights.mp4*


In [6]:
pose.process_video(pose.PifPafModel.fast, Path(pose_data, 'basketball.jl'), 
                   Path(videos, 'basketball.mp4'), skip_frame=10)

INFO:RTS:Processing /mnt/d/Videos/EPFL/basketball.mp4


True

In [ ]:
keypoints = data_json['data']['annotations'][0]['keypoints']

# Reshape keypoints into list of tuples (x, y, confidence)
keypoints = [(keypoints[i], keypoints[i+1], keypoints[i+2]) for i in range(0, len(keypoints), 3)]

# Associations
associations = [
    (15, 13),  # left_ankle --> left_knee
    (13, 11),  # left_knee --> left_hip
    (16, 14),  # right_ankle --> right_knee
    (14, 12),  # right_knee --> right_hip
    (11, 12),  # left_hip --> right_hip
    (5, 11),   # left_shoulder --> left_hip
    (6, 12),   # right_shoulder --> right_hip
    (5, 6),    # left_shoulder --> right_shoulder
    (5, 7),    # left_shoulder --> left_elbow
    (6, 8),    # right_shoulder --> right_elbow
    (7, 9),    # left_elbow --> left_wrist
    (8, 10),   # right_elbow --> right_wrist
    (1, 2),    # left_eye --> right_eye
    (0, 1),    # nose --> left_eye
    (0, 2),    # nose --> right_eye
    (1, 3),    # left_eye --> left_ear
    (2, 4),    # right_eye --> right_ear
    (3, 5),    # left_ear --> left_shoulder
    (4, 6)     # right_ear --> right_shoulder
]


def compute_angle_np_adjusted(p1, p2, p3):
    """
    Compute angle at p2 given points p1, p2, and p3 using NumPy.
    Floor to 0 if the angle is below 1 degree.
    """
    # Create vectors
    vec1 = np.array([p1[0] - p2[0], p1[1] - p2[1]])
    vec2 = np.array([p3[0] - p2[0], p3[1] - p2[1]])
    
    # Compute dot product and magnitudes
    dot_product = np.dot(vec1, vec2)
    mag1 = np.linalg.norm(vec1)
    mag2 = np.linalg.norm(vec2)
    
    # Compute angle in radians
    angle_rad = np.arccos(dot_product / (mag1 * mag2))
    
    # Convert to degrees
    angle_deg = np.degrees(angle_rad)
    
    # Floor to 0 if close to 0 (below 1 degree)
    if angle_deg < 1:
        angle_deg = 0.0

    return angle_deg

# Compute angles for all associations using NumPy with adjusted tolerance
angles_adjusted = {}
for i, (start, end) in enumerate(associations[:-1]):  # Exclude last association for now
    next_end = associations[i + 1][1]  # Assume next association shares the 'end' point
    if start == -1 or end == -1 or next_end == -1:
        continue  # Skip if any point is missing
    if keypoints[start][2] == 0 or keypoints[end][2] == 0 or keypoints[next_end][2] == 0:
        continue  # Skip if any point has zero confidence
    angle = compute_angle_np_adjusted(keypoints[start], keypoints[end], keypoints[next_end])
    angles_adjusted[f"{start} --> {end} --> {next_end}"] = angle